# Word Embedding for Sequence Processing

**The goal of this practical is to use pre-trained word embedding for adressing the sequence prediction tasks studied in week 2: PoS and chunking.**

In [1]:
import numpy as np
import gensim.downloader as api
from gensim.models import KeyedVectors

## 0) Loading PoS (or chunking) datasets (small or large)

In [2]:
def load(filename):
    listeDoc = list()
    with open(filename, "r") as f:
        doc = list()
        for ligne in f:
            #print "l : ",len(ligne)," ",ligne
            if len(ligne) < 2: # fin de doc
                listeDoc.append(doc)
                doc = list()
                continue
            mots = ligne.replace("\n","").split(" ")
            doc.append((mots[0],mots[2])) # mettre mots[2] à la place de mots[1] pour le chuncking
    return listeDoc

In [3]:
bSmall = False

if(bSmall==True):
    filename = "datasets/conll2000/chtrain.txt" 
    filenameT = "datasets/conll2000/chtest.txt" 

else:
    # Larger corpus .
    filename = "datasets/conll2000/train.txt" 
    filenameT = "datasets/conll2000/test.txt" 

alldocs = load(filename)
alldocsT = load(filenameT)

print(len(alldocs)," docs read")
print(len(alldocsT)," docs (T) read")

8936  docs read
2012  docs (T) read


# 1) Word embedding for classifying each word

### Pre-trained word2vec

In [4]:
import gensim.downloader as api
bload = False
fname = "word2vec-google-news-300"
sdir = "word2vec-google-news-300/" # Change

if(bload==True):
    wv_pre_trained = KeyedVectors.load(sdir+fname+".dat")
else:    
    wv_pre_trained = api.load(fname)
    wv_pre_trained.save(sdir+fname+".dat")

### Some token on the dataset are missing, we will encode them with a random vector
This is sub-optimal, but we need to do something

In [5]:
def randomvec():
    default = np.random.randn(300)
    default = default  / np.linalg.norm(default)
    return default

In [6]:
np.random.seed(seed=10) # seed the randomness

dictadd = dict()
cpt=0
for d in alldocs:
    cpt+=1
    print(" ****** Document ******",cpt)
    for (x,pos) in d:
        if (not (x in wv_pre_trained) and not (x in dictadd)):
            print(x," not in WE, adding it with random vector")
            dictadd[x] = randomvec()
            
for d in alldocsT:
    cpt+=1
    print(" ****** TEST Document ******",cpt)
    for (x,pos) in d:
        if (not (x in wv_pre_trained) and not (x in dictadd)):
            print(x," not in WE, adding it with random vector")
            dictadd[x] = randomvec()
            #wv_pre_trained.add_vector(x,randomvec())
            

 ****** Document ****** 1
to  not in WE, adding it with random vector
,  not in WE, adding it with random vector
a  not in WE, adding it with random vector
and  not in WE, adding it with random vector
's  not in WE, adding it with random vector
near-record  not in WE, adding it with random vector
.  not in WE, adding it with random vector
 ****** Document ****** 2
of  not in WE, adding it with random vector
 ****** Document ****** 3
 ****** Document ****** 4
16  not in WE, adding it with random vector
15  not in WE, adding it with random vector
 ****** Document ****** 5
``  not in WE, adding it with random vector
''  not in WE, adding it with random vector
 ****** Document ****** 6
 ****** Document ****** 7
-LRB-  not in WE, adding it with random vector
3.2  not in WE, adding it with random vector
-RRB-  not in WE, adding it with random vector
 ****** Document ****** 8
2.2  not in WE, adding it with random vector
2.3  not in WE, adding it with random vector
1988  not in WE, adding it w

 ****** Document ****** 6166
JROE  not in WE, adding it with random vector
 ****** Document ****** 6167
 ****** Document ****** 6168
6:50  not in WE, adding it with random vector
 ****** Document ****** 6169
CAROLG  not in WE, adding it with random vector
 ****** Document ****** 6170
 ****** Document ****** 6171
 ****** Document ****** 6172
 ****** Document ****** 6173
 ****** Document ****** 6174
7:13  not in WE, adding it with random vector
 ****** Document ****** 6175
 ****** Document ****** 6176
 ****** Document ****** 6177
 ****** Document ****** 6178
8:01  not in WE, adding it with random vector
 ****** Document ****** 6179
 ****** Document ****** 6180
 ****** Document ****** 6181
 ****** Document ****** 6182
 ****** Document ****** 6183
 ****** Document ****** 6184
 ****** Document ****** 6185
 ****** Document ****** 6186
 ****** Document ****** 6187
9:31  not in WE, adding it with random vector
 ****** Document ****** 6188
 ****** Document ****** 6189
 ****** Document ****** 61

### Add the (key-value) 'random' word embeddings for missing inputs

In [7]:
## YOUR CODE HERE
wv_pre_trained.add_vectors(list(dictadd.keys()), list(dictadd.values()))

### Store the train and test datasets: a word embedding for each token in the sequences

In [8]:
wvectors = [wv_pre_trained[word] for doc in alldocs for word, tag in doc]
wvectorsT = [wv_pre_trained[word] for doc in alldocsT for word, tag in doc]

### Check the size of your train/test datasets

In [9]:
len(wvectors), len(wvectorsT)
## YOUR CODE HERE

(211727, 47377)

### Collecting train/test labels

In [10]:
# Labels train/test

buf2 = [[pos for m,pos in d ] for d in alldocs]
cles = []
[cles.extend(b) for b in buf2]
cles = np.unique(np.array(cles))
cles2ind = dict(zip(cles,range(len(cles))))
nCles = len(cles)
print(nCles," keys in the dictionary")

labels  = np.array([cles2ind[pos] for d in alldocs for (m,pos) in d ])
#np.array([cles2ind[pos] for (m,pos) in d for d in alldocs])
labelsT  = np.array([cles2ind.setdefault(pos,len(cles)) for d in alldocsT for (m,pos) in d ])

print(len(cles2ind)," keys in the dictionary")

22  keys in the dictionary
23  keys in the dictionary


In [11]:
print(labels.shape)
print(labelsT.shape)

(211727,)
(47377,)


### Train a Logistic Regression Model! 
**An compare performances to the baseline and sequence models (HMM/CRF) or practical 2a**

In [12]:
## YOUR CODE HERE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lr = LogisticRegression()
lr.fit(wvectors, labels)

labelsP = lr.predict(wvectorsT)
accuracy_score(labelsP, labelsT)

/Users/aylinsoykok/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7718724275492328

# 2) Using word embedding with CRF

## We will define the following features functions for CRF

In [13]:
def features_wv(sentence, index):
    v = wv_pre_trained.get_vector(sentence[index])
    d = {'f'+str(i):v[i] for i in range(300)}
    return d

def features_structural(sentence, index):
    return {
        'word': sentence[index],
        'is_first': index == 0,
        'is_last': index == len(sentence) - 1,
        'is_capitalized': sentence[index][0].upper() == sentence[index][0],
        'is_all_caps': sentence[index].upper() == sentence[index],
        'is_all_lower': sentence[index].lower() == sentence[index],
        'prefix-1': sentence[index][0],
        'prefix-2': sentence[index][:2],
        'prefix-3': sentence[index][:3],
        'suffix-1': sentence[index][-1],
        'suffix-2': sentence[index][-2:],
        'suffix-3': sentence[index][-3:],
        'prev_word': '' if index == 0 else sentence[index - 1],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1],
        'has_hyphen': '-' in sentence[index],
        'is_numeric': sentence[index].isdigit(),
     ## We will define the following features functions for CRF## We will define the following features functions for CRF   'capitals_inside': sentence[index][1:].lower() != sentence[index][1:]
    }
def features_wv_plus_structural(sentence, index):
    v = wv_pre_trained.get_vector(sentence[index]) 
    d = {'f'+str(i):v[i] for i in range(300)}

    return {**d, **features_structural(sentence, index)}

## [Question]: explain what the 3 feature functions encode and what their differences are

features_wv : A partir d'une phrase et une indice, extrait les caractéristiques basées sur les word embeddings de chaque mot dans une phrase avec un modèle prétrainé. Elle retourne un dictionnaire contenant les caractéristiques des mots selon son indice représentés par un vecteur de 300 dimensions. 

features_structural : En prénant en entrée la phrase et son indice, celle-ci extrait les caractéristiques structurelles des mots de cette phrase. Elle retourne les caractéristiques comme si l'indice est première ou la dernière, si le mot est en majuscules etc.

features_wv_plus_structural : Cette fonction est la combinaison des 2 fonctions en dessus. Grâce à un modèle de WE prétrainé, elle extrait les caractéristiques de base et structurelles. Elle retourne une dictionnaire contenant ces deux caractéristiques.

### You can now train a CRF with the 3 features and analyse the results

In [16]:
import warnings
warnings.filterwarnings('ignore')

from nltk.tag.crf import CRFTagger

funcs = [features_wv,features_structural,features_wv_plus_structural]

for func in funcs:
    crf = CRFTagger(feature_func=func)
    ## Train the model
    crf.train(alldocs, 'model_w2v_crf_1')
    ## Evaluate performances
    perf = crf.evaluate(alldocsT)
    print("Performance de ",func.__name__,":",perf)


Performance de  features_wv : 0.8817147561052832
Performance de  features_structural : 0.9384089326044283
Performance de  features_wv_plus_structural : 0.9452054794520548
